# Model Training and Evaluation - GPS Spoofing Detection

This notebook demonstrates model training and evaluation for GPS spoofing detection.

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from utils.synthetic_data import generate_synthetic_dataset
from preprocessing.signal_processing import generate_ca_code
from features.pipeline import build_feature_vector, preprocess_features
from models.train import train_model, evaluate_model, print_evaluation_report
from models.persistence import save_model, load_model
from utils.plots import plot_confusion_matrix, plot_roc_curves

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Generate Dataset and Extract Features

In [ ]:
# Generate synthetic signals (larger dataset for training)
signals, labels, metadata = generate_synthetic_dataset(
    num_authentic=200,
    num_spoofed=200,
    fs=5e6,
    duration=0.5,
    prn_range=(1, 5),
    random_state=42
)

print(f"Generated {len(signals)} signals")
print(f"Class distribution: {np.bincount(labels)}")

In [ ]:
# Extract features
print("Extracting features...")

all_features = []
for i, signal in enumerate(signals):
    if i % 50 == 0:
        print(f"  Processing signal {i+1}/{len(signals)}...")
    
    prn = metadata[i]['prn']
    ca_code = generate_ca_code(prn)
    
    features = build_feature_vector(
        signal=signal,
        prn_code=ca_code,
        fs=5e6,
        label=labels[i],
        metadata={'prn': prn, 'segment_index': i}
    )
    all_features.append(features)

df_features = pd.DataFrame(all_features)
print(f"\nFeature extraction complete. Shape: {df_features.shape}")

## 2. Prepare Data for Training

In [ ]:
# Separate features and labels
X = df_features.drop(columns=['label', 'prn', 'segment_index'], errors='ignore')
y = df_features['label'].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"\nTraining set class distribution: {np.bincount(y_train)}")
print(f"Test set class distribution: {np.bincount(y_test)}")

In [ ]:
# Preprocess features
X_train_processed, imputer, scaler, _ = preprocess_features(X_train, y_train, fit=True)
X_test_processed, _, _, _ = preprocess_features(X_test, y_test, imputer=imputer, scaler=scaler, fit=False)

print(f"Preprocessed training shape: {X_train_processed.shape}")
print(f"Preprocessed test shape: {X_test_processed.shape}")

## 3. Train Random Forest Model (Baseline)

Train Random Forest with balanced class weights.

In [ ]:
# Train Random Forest
print("Training Random Forest...")
rf_model, rf_train_metrics = train_model(
    X_train_processed, y_train,
    model_name='random_forest',
    params={
        'n_estimators': 100,
        'max_depth': 15,
        'min_samples_split': 5,
        'class_weight': 'balanced',
    },
    use_smote=False,
    cv=5,
    random_state=42
)

print("\nTraining Metrics:")
print(f"CV Mean Accuracy: {rf_train_metrics['cv_mean']:.4f} (+/- {rf_train_metrics['cv_std']:.4f})")
print(f"Training Accuracy: {rf_train_metrics['train_accuracy']:.4f}")

## 4. Evaluate Random Forest

In [ ]:
# Evaluate on test set
rf_metrics = evaluate_model(rf_model, X_test_processed, y_test)

# Print report
print_evaluation_report(rf_metrics)

In [ ]:
# Plot confusion matrix
plot_confusion_matrix(
    np.array(rf_metrics['confusion_matrix']),
    class_names=['Authentic', 'Spoofed'],
    title='Random Forest - Confusion Matrix'
)
plt.show()

## 5. Train with SMOTE (Optional)

Try training with SMOTE oversampling.

In [ ]:
# Train Random Forest with SMOTE
print("Training Random Forest with SMOTE...")
rf_smote_model, rf_smote_train_metrics = train_model(
    X_train_processed, y_train,
    model_name='random_forest',
    params={
        'n_estimators': 100,
        'max_depth': 15,
        'class_weight': 'balanced',
    },
    use_smote=True,
    cv=5,
    random_state=42
)

print("\nTraining Metrics (with SMOTE):")
print(f"CV Mean Accuracy: {rf_smote_train_metrics['cv_mean']:.4f} (+/- {rf_smote_train_metrics['cv_std']:.4f})")
print(f"Training Accuracy: {rf_smote_train_metrics['train_accuracy']:.4f}")

# Evaluate
rf_smote_metrics = evaluate_model(rf_smote_model, X_test_processed, y_test)
print_evaluation_report(rf_smote_metrics)

## 6. Compare Multiple Models

In [ ]:
# Train SVM
print("Training SVM...")
svm_model, svm_train_metrics = train_model(
    X_train_processed, y_train,
    model_name='svm',
    params={'C': 1.0, 'gamma': 'scale'},
    cv=3,  # Fewer folds for SVM (slower)
    random_state=42
)

svm_metrics = evaluate_model(svm_model, X_test_processed, y_test)
print(f"SVM Test Accuracy: {svm_metrics['accuracy']:.4f}")

In [ ]:
# Compare models
comparison = pd.DataFrame({
    'Model': ['Random Forest', 'RF + SMOTE', 'SVM'],
    'Accuracy': [
        rf_metrics['accuracy'],
        rf_smote_metrics['accuracy'],
        svm_metrics['accuracy']
    ],
    'Precision': [
        rf_metrics['precision'],
        rf_smote_metrics['precision'],
        svm_metrics['precision']
    ],
    'Recall': [
        rf_metrics['recall'],
        rf_smote_metrics['recall'],
        svm_metrics['recall']
    ],
    'F1 Score': [
        rf_metrics['f1_score'],
        rf_smote_metrics['f1_score'],
        svm_metrics['f1_score']
    ],
    'ROC AUC': [
        rf_metrics['roc_auc'],
        rf_smote_metrics['roc_auc'],
        svm_metrics['roc_auc']
    ]
})

print("\nModel Comparison:")
print("="*80)
print(comparison.to_string(index=False))

In [ ]:
# Plot ROC curves
y_scores = {
    'Random Forest': rf_model.predict_proba(X_test_processed)[:, 1],
    'RF + SMOTE': rf_smote_model.predict_proba(X_test_processed)[:, 1],
    'SVM': svm_model.predict_proba(X_test_processed)[:, 1]
}

plot_roc_curves(y_test, y_scores, title='Model Comparison - ROC Curves')
plt.show()

## 7. Save Best Model

In [ ]:
# Select best model based on F1 score
best_idx = comparison['F1 Score'].idxmax()
best_model_name = comparison.loc[best_idx, 'Model']

print(f"Best model: {best_model_name}")
print(f"F1 Score: {comparison.loc[best_idx, 'F1 Score']:.4f}")

# Save the best model (Random Forest in this case)
model_path = '../data/processed/best_model.pkl'
metadata = {
    'model_name': best_model_name,
    'metrics': rf_metrics if best_idx == 0 else (rf_smote_metrics if best_idx == 1 else svm_metrics),
    'features': list(X.columns),
    'random_state': 42
}

best_model = rf_model if best_idx == 0 else (rf_smote_model if best_idx == 1 else svm_model)
save_model(best_model, model_path, metadata)

# Also save preprocessors
import joblib
joblib.dump(imputer, '../data/processed/imputer.pkl')
joblib.dump(scaler, '../data/processed/scaler.pkl')
print("Preprocessors saved.")

## 8. Test Loading and Inference

In [ ]:
# Load model
loaded_model, loaded_metadata = load_model(model_path)

print("\nLoaded model metadata:")
print(f"Model: {loaded_metadata['model_name']}")
print(f"Test Accuracy: {loaded_metadata['metrics']['accuracy']:.4f}")

# Test prediction
sample_idx = 0
X_sample = X_test_processed[sample_idx:sample_idx+1]
y_sample = y_test[sample_idx]

prediction = loaded_model.predict(X_sample)[0]
probability = loaded_model.predict_proba(X_sample)[0]

print(f"\nTest sample prediction:")
print(f"True label: {'Spoofed' if y_sample == 1 else 'Authentic'}")
print(f"Predicted: {'Spoofed' if prediction == 1 else 'Authentic'}")
print(f"Probabilities: Authentic={probability[0]:.3f}, Spoofed={probability[1]:.3f}")

## Summary

**Best Model Performance:**
- Model: Random Forest with balanced class weights
- Achieves high accuracy on synthetic data
- Key features: peak_to_secondary, cn0_estimate, fpw

**Recommendations:**
1. Test on real GPS spoofing datasets (FGI-SpoofRepo, TEXBAT)
2. Fine-tune hyperparameters for specific scenarios
3. Consider ensemble methods for improved robustness
4. Monitor false alarm rate in deployment

**Next Steps:**
- Deploy model using `scripts/script_run_pipeline.py`
- Integrate with real-time GPS receiver
- Collect more diverse spoofing scenarios for training